The transfer function of the causal LTI system with a realization $(A, B, C, D)$ is given by
\begin{equation*}
    H(s) = C(sI_{n} - A)^{-1}B.
\end{equation*}
Notice that $H(s) = Cv(s)$, where $v(s)$ is the solution of a parametrized linear coercive model
\begin{equation}
    a(v, w; s) = l(w),
\end{equation}
where $a(v, w; s) = w^{*}(sI_{n} - A)v$ and $l(w) = w^{*}B$. 

We replace the matrix computation of $(sI_{n} - A)^{-1}B$ with a parametrized model (1), and the transfer function $H(s)$ is the output of the model, with $C$ serving as the output operator.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pymor.basic import *
from pymor.algorithms.to_matrix import to_matrix
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.operators.constructions import LincombOperator
from pymor.parameters.functionals import ProjectionParameterFunctional

In [2]:
def TF_exact(A,B,C,s):

    # NumPy convertions
    A = A if isinstance(A, np.ndarray) else to_matrix(A).toarray()
    B = B if isinstance(B, np.ndarray) else to_matrix(B)
    C = C if isinstance(C, np.ndarray) else to_matrix(C)
    
    tf = (C@np.linalg.inv(s*np.eye(A.shape[0]) - A)@B)

    return tf

In [2]:
def MatrixModel(A, B, C):

    '''
    Description
    -----------
    This function creates a StationaryModel for the following linear coercive model derived from the given three matrices A, B, and C:
    
        a(v, w; s) = w*(sI_n - A)v and l(w) = w*B.

    Parameters
    ----------
    A: NumpyMatrixOperator or numpy.ndarray
        A.shape = (n, n) or to_matrix(A).shape = (n, n)
    B: NumpyMatrixOperator or numpy.ndarray
        A.shape = (n, 1) or to_matrix(B).shape = (n, 1)
    C: NumpyMatrixOperator or numpy.ndarray
        A.shape = (1, n) or to_matrix(C).shape = (1, n)

    Return
    ------
    model_TF: StationaryModel
    '''

    # Define operators (and also a dimension of a model)
    if isinstance(A, np.ndarray):
        dim = A.shape[0]
        A_op = NumpyMatrixOperator(A)
    else:
        dim = to_matrix(A).shape[0]
        A_op = A

    if isinstance(B, np.ndarray):
        B_op = NumpyMatrixOperator(B) 
    else:
        B_op = B  

    if isinstance(C, np.ndarray):
        C_op = NumpyMatrixOperator(C)  
    else:
        C_op = C

    I_op = NumpyMatrixOperator(np.eye(dim))
    
    # Define parameter functional for 's'
    s_param = ProjectionParameterFunctional('s', 1)

    # Define bilinear form a(v, w; s) = w*(sI - A)v
    a_op_1 = LincombOperator([I_op, A_op], [s_param, -1])

    # Define linear functional l(w) = w^*B
    l_op_1 = B_op

    # Define the StationaryModel
    model_TF = StationaryModel(operator=a_op_1, rhs=l_op_1, output_functional = C_op)

    return model_TF


### Examples
Notice that to make this process more convenient, we provide two options for input matrices. If one has the matrices as `NumPy` arrays, they can import them directly without changing their type to work with `pyMOR`. Additionally, if one wants to input matrices from existing `pyMOR` models, this can also be done directly. To demonstrate this, we provide two examples: one with random matrices that we construct ourselves, and the other with matrices imported from the `penzl_example` in `pymor.models.examples`.

In [ ]:
# StationaryModel constructed using random numpy arrays
matrixA = np.random.rand(20, 20)
matrixB = np.random.rand(20).reshape(20,1)
matrixC = np.random.rand(20).reshape(1,20)

# StationaryModel
model_TF = MatrixModel(A = matrixA, B = matrixB, C = matrixC)
model_TF

In [ ]:
# Set a value for evaluation
parameter = 1.4 + 2*1j  # s = 1.4 + 2i

# Solve the model - (sI_n - A)^{-1}B
solution = model_TF.solve(parameter).to_numpy()

# Get the output - C(sI_n - A)^{-1}B
output = model_TF.output(parameter)

# Exact transfer function
exact = TF_exact(matrixA, matrixB, matrixC, parameter)

# A comparison between the model's result and the exact computation
print(f'The output of the model is {output[0,0]}.')
print(f'The exact value of a transfer function H{parameter} is {exact[0,0]}.')
print(f'|H(s) - H_m(s)| is {abs(exact - output)[0,0]}.')

In [ ]:
# StationaryModel constructed using matrices obtained from penzl example
from pymor.models.examples import penzl_example

penzl = penzl_example()

model_penzl = MatrixModel(penzl.A, penzl.B, penzl.C)
model_penzl

In [ ]:
# Set a value for evaluation
parameter = -1 + 7*1j  # s = - 1 + 7i

# Solve the model - (sI_n - A)^{-1}B
solution = model_penzl.solve(parameter).to_numpy()

# Get the output - C(sI_n - A)^{-1}B
output = model_penzl.output(parameter)

# Exact transfer function
exact = TF_exact(penzl.A, penzl.B, penzl.C, parameter)

# A comparison between the model's result and the exact computation
print(f'The output of the model is {output[0,0]}.')
print(f'The exact value of a transfer function H{parameter} is {exact[0,0]}.')
print(f'|H(s) - H_m(s)| is {abs(exact - output)[0,0]}.')

### Caution!
Note that for the MIMO case, an error arises when we construct the `StationaryModel` because `rhs.source.is_scalar` is False. This occurs because $v$ and $w$ are matrices, not vectors.

In [ ]:
# StationaryModel constructed using random numpy arrays
matrixA = np.random.rand(20, 20)
matrixB = np.random.rand(40).reshape(20,2)
matrixC = np.random.rand(60).reshape(3,20)

# StationaryModel
model_TF = MatrixModel(A = matrixA, B = matrixB, C = matrixC)
model_TF

#### Remedy
The right question would be whether it is possible to split the model into submodels with `rhs.source.is_scalar = True`, or in other words, into models with \( m = p = 1 \). In this context, one would need to check if it is possible to partition the MIMO model into SISO models. It turns out that this is indeed possible, and several methodologies exist for doing so. One commonly used method to split a MIMO model into SISO models is Singular Value Decomposition (SVD). In this discussion, we will not focus on the MIMO case, as SVD can be employed to obtain several SISO models, and we can apply the method we proposed for each individual SISO.

## POD-Galerkin RB method

Let $A \in \mathbb{R}^{n \times n}$, $B \in \mathbb{R}^{n \times 1}$, and $C \in \mathbb{R}^{1 \times n}$, and let $\mathcal{P}$ be the set of admissible parameters for the linear coercive FOM described above. The proposed approach is to approximate the transfer function over the parameter space $\mathcal{P}$ using ROM evaluations instead of exact matrix computations. For model reduction, we employ the POD-Galerkin RB method. It is important to note that this approximation is effective if the parameter space $\mathcal{P}$ is known, as the snapshot matrix $S$, whose column entries are snapshot solutions ($S[:,i] = (s_{i}I - A)^{-1}B$), is constructed based on parameters within the given parameter space $\mathcal{P}$. 

To demonstrate the proposed method, we will use `penzl_example` from `pymor.models.examples`.

In [36]:
from pymor.models.examples import penzl_example

penzl = penzl_example()
model_penzl = MatrixModel(penzl.A, penzl.B, penzl.C)

In [ ]:
# Define a parameter space - P
parameter_space = model_penzl.parameters.space(0.01, 10.)

# Define a training set - snapshot parameters
training_set = parameter_space.sample_randomly(50)

# Compute FOM solutions for the parameters in the training set
solution_snapshots = model_penzl.solution_space.empty()
for s in training_set:
    solution_snapshots.append(model_penzl.solve(s))

# Snapshot matrix S - pyMOR framework
print(f'An {solution_snapshots.impl._array.shape[0]} by {solution_snapshots.impl._array.shape[1]} matrix (obtained through pyMOR) is \n {solution_snapshots}')

# Snapshot matrix S
snapshot_matrix = solution_snapshots.to_numpy().T
print(f'An {snapshot_matrix.shape[0]} by {snapshot_matrix.shape[1]} snapshot matrix is \n {snapshot_matrix}')

In [ ]:
## Construction of the reduced basis V_N using the pyMOR framework

# Define the total number of modes
modes = 30

# Find POD basis
pod_basis, pod_singular_values = pod(solution_snapshots, atol = 0, rtol = 0, modes = modes)

print(f'The reduced basis (containing the first {modes} left singular vectors (POD modes) of the snapshot matrix as its rows) is \n {pod_basis}') # Caution: POD modes are represented by its rows, not columns.

In [ ]:
## Construction of the reduced basis V_N using the NumPy framework

# Find the Singular Value Decomposition (SVD) of the snapshot matrix -> S = UΣV^H
U, D, Vt = np.linalg.svd(snapshot_matrix, full_matrices = True) 

# Define the total number of modes
modes_numpy = 20 

if modes_numpy > min(snapshot_matrix.shape):
    raise ValueError("modes cannot exceed the rank of the snapshot matrix.")

# Find POD basis (reduced basis)
pod_basis_numpy = U[:,:modes_numpy]

print(f'The reduced basis (containing the first {modes_numpy} left singular vectors (POD modes) of the snapshot matrix as its columns) is \n {pod_basis_numpy}') # basis of the reduced space == reduced basis

In [ ]:
## Remark: Select the number of modes according to the energy criterion : NumPy framework

'''
The number of modes m should be chosen based on the energy criterion, which ensures that a sufficient 
portion of the system's total energy (or variance) is captured by the first m modes.
'''

cumulative_energy = np.cumsum(D**2) / np.sum(D**2)

# Select the number of modes m to capture at least 99% of the energy
threshold = 0.99  # 99% of the total energy
m = np.argmax(cumulative_energy >= threshold) + 1

print(f'The number of modes required to capture at least 99% of the energy is {m}.') 

In [ ]:
from pymor.reductors.basic import StationaryRBReductor

# Galerkin projection of a StationaryModel
pod_reductor = StationaryRBReductor(model_penzl, RB = pod_basis) 
pod_rom = pod_reductor.reduce()

# ROM constructed using the POD-Galerkin reduced basis method
pod_rom

In [ ]:
## Example

# Set a value for evaluation
eval_parameter = -1 + 7*1j  # s = -1 + 7j

# Solve the reduced order model (ROM)
ROM_solution = pod_rom.solve(eval_parameter)

# Get the output of the reduced order model (ROM)
ROM_output = pod_rom.output(eval_parameter)

# Exact transfer function evaluated at a given value
exact_TF = TF_exact(penzl.A, penzl.B, penzl.C, eval_parameter)

# A comparison between the reduced model's result and the exact computation
print(f'The approximate value of the transfer function at {eval_parameter} is {ROM_output[0,0]}.')
print(f'The exact value of a transfer function H{eval_parameter} is {exact_TF[0,0]}.')
print(f'|H(s) - H_ROM(s)| is {abs(exact_TF - ROM_output)[0,0]}.')

### Offline-online decomposition for evaluating the transfer function

We will summarize the steps outlined above and analyze the computational cost to demonstrate the efficiency of the proposed method.

In [7]:
# Offline phase: Construct the reduced-order model (ROM)

def TFROM(model, rng, snap, rorder: int = None, atol: float = 0, rtol:float = 0):

    '''
    Description
    -----------
    This function returns the ROM required to obtain the approximated transfer function of a given LTI model using the POD-Galerkin reduced basis method.

    Parameters
    ----------
    model: LTIModel (SISO)
    rng: tuple
        The range of the parameter space for the linear coercive FOM
            Real-valued parameter space -> recommended choice
                rng = (.,.)
            Complex-valued parameter space -> It is advisable to use this method if you want to obtain the transfer function at a complex-valued parameter.
                The first two values represent the range for the real part of the parameters, while the last two represent the range for the imaginary part.
                rng = (.,.,.,.)
    snap: int
        The number of snapshots to be used for the reduced order model (ROM)
    rorder: int
        The order of the reduced order model (ROM)
    rtol: float
        Singular values of snapshot matrix smaller than this value multiplied by the largest singular value are ignored
    atol: float
        Singular values of snapshot matrix smaller than this value are ignored

    Return
    ------
    TF_ROM: StationaryModel
    '''

    assert isinstance(rng, tuple)

    # StationaryModel constructed using matrices obtained from model matrices
    model_TF = MatrixModel(A = model.A, B = model.B, C = model.C)

    # Define a training set - snapshot parameters
    if len(rng) == 2: # real-valued parameter space
        training_set = np.random.uniform(low = rng[0], high = rng[1], size=(snap,)) 
    elif len(rng) == 4: # complex-valued parameter space
        training_set = np.random.uniform(low = rng[0], high = rng[1], size=(snap,)) + 1j*np.random.uniform(low = rng[2], high = rng[3], size=(snap,))

    # Compute FOM solutions for the parameters in the training set
    snapshot_matrix = model_TF.solution_space.empty()
    for s in training_set:
        snapshot_matrix.append(model_TF.solve(s))
    
    # Find POD basis
    pod_basis, _ = pod(snapshot_matrix, modes = rorder, atol = atol, rtol = rtol)

    # Galerkin projection of a StationaryModel
    pod_reductor = StationaryRBReductor(model_TF, RB = pod_basis) 
    TF_ROM = pod_reductor.reduce()

    return TF_ROM

In [ ]:
## Offline phase: Construct the reduced-order model (ROM)

import time
from pymor.models.examples import penzl_example

# Start timing
start_time_offline = time.time()

# Define the LTI model
model = penzl_example()

# Define the range for the parameter space
rng = (0.1, 100.)

# Define the number of snapshots to be used for the reduced order model
snap = 50

# Get the reduced-order model (ROM)
TF_ROM = TFROM(model, rng, snap)

# End timing
end_time_offline = time.time()

# Compute the elapsed time
elapsed_time_offline = end_time_offline - start_time_offline

In [83]:
## Online phase: Evaluate the transfer function at a set of points using the reduced-order solution

# Start timing
start_time_online = time.time()

# Define an evaluation set
card_set = 200
eval_set = np.random.uniform(low = 1, high = 70, size=(card_set,)) + 1j*np.random.uniform(low = 1, high = 70, size=(card_set,))

# Evaluate the approximate transfer function at a set of evaluation points
H_app = np.array([TF_ROM.output(s)[0, 0] for s in eval_set])

# Start timing
end_time_online = time.time()

# Compute the elapsed time
elapsed_time_online = end_time_online - start_time_online

In [84]:
## Exact computation

# Start timing
start_time_exact = time.time()

# Find the exact value of the transfer function at a set of evaluation points
H_exact = np.array([TF_exact(model.A, model.B, model.C, i)[0, 0] for i in eval_set])

# Start timing
end_time_exact = time.time()

# Compute the elapsed time
elapsed_time_exact = end_time_exact - start_time_exact

In [ ]:
## Computational error and execution time analysis

# Error analysis
error = max(np.abs(H_exact - H_app))
print(f'max|H(s) - H_m(s)| is {error}.')

# Computational time analysis (execution time analysis)
print(f'Offline phase total computation time: {elapsed_time_offline:.10f} seconds.')
print(f'Online phase total computation time: {elapsed_time_online:.10f} seconds.')
print(f'Exact computation total time: {elapsed_time_exact:.10f} seconds.')


### Visualization of approximate transfer function values at different reduced orders

In [ ]:
from pymor.models.examples import penzl_example

# Define evaluation points
s1 = 17.2 - 3.5*1j
s2 = 6.1 + 49.85*1j

# Define the LTI model
model_penzl = penzl_example()

# Define the range for the parameter space
rng = (1, 150.)

# Define the number of snapshots to be used for the reduced order model
snap = 40

# Evaluate the exact transfer function at the given values
H_penzl_1 = TF_exact(model_penzl.A, model_penzl.B, model_penzl.C, s1)[0, 0] 
H_penzl_2 = TF_exact(model_penzl.A, model_penzl.B, model_penzl.C, s2)[0, 0] 

# Define the range of reduced orders to test and evaluate the approximate transfer function for each order
range_plot = 20
H_app_1 = np.array([TFROM(model = model_penzl, rng = rng, snap = snap, rorder = order).output(s1)[0, 0] for order in range(1, range_plot + 1)])
H_app_2 = np.array([TFROM(model = model_penzl, rng = rng, snap = snap, rorder = order).output(s2)[0, 0] for order in range(1, range_plot + 1)])

# Compute the absolute error between the exact transfer function and the approximate values
error1 = np.abs(H_penzl_1 - H_app_1)
error2 = np.abs(H_penzl_2 - H_app_2)

In [ ]:
# Error norm plot
fig, ax = plt.subplots(figsize=(16, 7), dpi=100)
ax.semilogy(np.arange(1, range_plot + 1), error1, color='red', lw = 0.8, marker = 'x', label = s1)
ax.semilogy(np.arange(1, range_plot + 1), error2, color='blue', lw = 0.8, marker = 'x', label = s2)
ax.set_title("Error norms")
ax.set_xlabel("Reduced order")
ax.set_ylabel("Absolute error norm (log scale)")
ax.legend()

# Show the plot
plt.show()